In [1]:
import numpy as np
import pandas as pd
import os

In [11]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

In [3]:
os.getcwd()

'D:\\Desktop\\SUTD\\Term 8\\50.021 AI\\Final Project\\zhixuanRF'

In [4]:
df = pd.read_csv("numerical_data.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20112480 entries, 0 to 20112479
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   #followers          int64  
 1   #friends            int64  
 2   #favorites          int64  
 3   Retweets            int64  
 4   yr_sin              float64
 5   yr_cos              float64
 6   time_sin            float64
 7   time_cos            float64
 8   nMentions           int64  
 9   nHashtags           int64  
 10  nURLs               int64  
 11  positive_sentiment  int64  
 12  negative_sentiment  int64  
 13  Retweeted           int64  
dtypes: float64(4), int64(10)
memory usage: 2.1 GB


In [16]:
# retweeted = df["Retweeted"] > 0

In [17]:
df["log(Retweets)"] = np.log(df["Retweets"])

c:\python39\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
y = df["Retweets"]
df.drop(columns=["Retweeted", "Retweets"], inplace=True)
X = df

In [7]:
_, X_test, _, y_test = train_test_split(X, y, random_state=42)
X_test = MinMaxScaler().fit_transform(X_test)

In [9]:
from joblib import load
clf = load('RFClassifier.joblib') 
rgr = load('RFRegressorLog.joblib')

def model(X):
    retweeted = clf.predict(X)    
    retweeted_indices = np.argwhere(retweeted > 0)    
    retweeted_indices = retweeted_indices.ravel()
    
    numRetweets = rgr.predict(X[retweeted_indices,:])
    numRetweets = np.exp(numRetweets)
    
    retweeted = retweeted.astype(float)
    for index, numRetweet in zip(retweeted_indices, numRetweets):
        retweeted[index] = numRetweet
        
    return retweeted

In [ ]:
y_hat = model(X_test)

In [14]:
y_test_arr = y_test.values

In [17]:
# Sanity check on the 
y_hat[y_hat == 0]

array([], dtype=float64)

In [15]:
msle = mean_squared_log_error(y_test_arr, y_hat)
print(f"The MSE loss for the overall model is {msle}")

The MSE loss for the overall model is 1.6290576753957389
